In [27]:
import helpers
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import bz2
import json
import os
from importlib import reload

In [25]:
df = helpers.get_samples(filename="climate_highprob2020.json.bz2",num_samples=5000, random=False )


In [26]:
for i in range(10):
    print(df.quotation.iloc[i])

[ The delay ] will have an impact [ on Slough ] but that might be mitigated by the fact we are going to have this Western Rail Link to Heathrow. It looks like that may come in sooner than Crossrail.
[ The scheme ] treats addiction as an illness and the results so far have been extremely encouraging.
[ These ] actions will allow households who have an FHA-insured mortgage to meet the challenges of COVID-19 without fear of losing their homes, and help steady market concerns,
11 straight weeks of pre-season,
2019 was a landmark year for Fiverr as we completed a successful IPO, expanded the Fiverr ecosystem with new products, increased our international reach, and most importantly, continued our extraordinary growth momentum and march toward profitability,
7pm is when most hospitals change shifts. That's why. We love you heroes.
A collaborative opioid stewardship program, actively supported by pharmacists, will allow us to improve, monitor and evaluate opioid use and increase safety and ef

In [54]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Harold\AppData\Roaming\nltk_data...


True

In [29]:
for i in range(1):
    print(helpers.preprocess_quote(df.quotation.iloc[i]))

['delay', 'impact', 'slough', 'might', 'mitigated', 'fact', 'going', 'western', 'rail', 'link', 'heathrow', 'looks', 'like', 'may', 'sooner', 'crossrail', 'delay impact', 'impact slough', 'slough might', 'might mitigated', 'mitigated fact', 'fact going', 'going western', 'western rail', 'rail link', 'link heathrow', 'heathrow looks', 'looks like', 'like may', 'may sooner', 'sooner crossrail']


In [56]:
text = df.quotation.iloc[1]

In [57]:
text

'[ The scheme ] treats addiction as an illness and the results so far have been extremely encouraging.'

In [59]:
# Following this tutorial: https://realpython.com/python-nltk-sentiment-analysis/#:~:text=Sentiment%20analysis%20is%20the%20practice,obtain%20insights%20from%20linguistic%20data.

#NLTK already has a built-in, pretrained sentiment analyzer called VADER (Valence Aware Dictionary and sEntiment Reasoner).

#Since VADER is pretrained, you can get results more quickly than with many other analyzers.
#  However, VADER is best suited for language used in social media, like short sentences with some slang and abbreviations.
# It’s less accurate when rating longer, structured sentences, but it’s often a good launching point.


from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(text)

{'neg': 0.132, 'neu': 0.687, 'pos': 0.181, 'compound': 0.2484}